Final Output

The script does not work for cars where the full brand name is not used in the car's name. For example:   ̶F̶a̶b̶r̶y̶k̶a̶ ̶Sa̶m̶o̶c̶h̶o̶d̶ó̶w̶ ̶Os̶o̶b̶o̶w̶y̶c̶h̶   FSO Polonez,  ̶F̶o̶r̶d̶ ̶M̶o̶t̶o̶r̶ ̶C̶o̶m̶p̶a̶n̶y̶ Ford Focus Mk3. Working inputs "Nissan Skyline" "Opel Astra G" "Audi A6 C5" "Citroën C5"

> Add blockquote



---



In [ ]:
import os
import requests

# SPARQL endpoints
dbpedia_endpoint = "https://dbpedia.org/sparql"
wikidata_endpoint = "https://query.wikidata.org/sparql"

def get_wikidata_uri_and_abstract(entity_name):
    """Fetch the Wikidata URI and abstract for a given entity from DBpedia."""
    sparql_query = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    SELECT ?wikidataUri ?abstract WHERE {{
      dbr:{entity_name} owl:sameAs ?wikidataUri .
      dbr:{entity_name} dbo:abstract ?abstract .
      FILTER(STRSTARTS(STR(?wikidataUri), "http://www.wikidata.org/entity/"))
      FILTER(LANG(?abstract) = "en")
    }}
    LIMIT 1
    """
    response = requests.get(dbpedia_endpoint, params={'query': sparql_query, 'format': 'json'})

    if response.status_code != 200:
        raise Exception(f"Error querying DBpedia: {response.status_code} - {response.text}")

    data = response.json()

    if data['results']['bindings']:
        wikidata_uri = data['results']['bindings'][0]['wikidataUri']['value']
        abstract = data['results']['bindings'][0]['abstract']['value']
        return wikidata_uri, abstract
    else:
        return None, f"No Wikidata URI or abstract found for {entity_name}"

def get_logo_from_wikidata(wikidata_uri):
    """Fetch the logo URL from Wikidata for a given entity URI."""
    entity_id = wikidata_uri.split('/')[-1]

    sparql_query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT ?logo WHERE {{
      wd:{entity_id} wdt:P154 ?logo .
    }}
    LIMIT 1
    """

    response = requests.get(wikidata_endpoint, params={'query': sparql_query, 'format': 'json'})

    if response.status_code != 200:
        raise Exception(f"Error querying Wikidata: {response.status_code} - {response.text}")

    data = response.json()

    if data['results']['bindings']:
        logo_url = data['results']['bindings'][0]['logo']['value']
        return logo_url
    else:
        return "No logo found"

def get_flag_from_wikidata(wikidata_uri):
    """Fetch the country flag URL from Wikidata for the manufacturer of the entity."""
    entity_id = wikidata_uri.split('/')[-1]

    sparql_query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT ?country ?flag WHERE {{
      wd:{entity_id} wdt:P17 ?country .   # Manufacturer country
      ?country wdt:P41 ?flag .           # Country flag image
    }}
    LIMIT 1
    """

    response = requests.get(wikidata_endpoint, params={'query': sparql_query, 'format': 'json'})

    if response.status_code != 200:
        raise Exception(f"Error querying Wikidata: {response.status_code} - {response.text}")

    data = response.json()

    if data['results']['bindings']:
        flag_url = data['results']['bindings'][0]['flag']['value']
        return flag_url
    else:
        return "No flag found"

def get_car_image(label):
    """Fetch the image URL for a car model from Wikidata."""
    sparql_query = f"""
    SELECT ?image WHERE {{
      ?car rdfs:label "{label}"@en;
           wdt:P31 wd:Q3231690.  # Instance of automobile model
      OPTIONAL {{ ?car wdt:P18 ?image. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 1
    """
    print(label)
    response = requests.get(wikidata_endpoint, params={"query": sparql_query, "format": "json"})

    if response.status_code != 200:
        raise Exception(f"Error querying Wikidata: {response.status_code} - {response.text}")

    data = response.json()

    if 'results' in data and 'bindings' in data['results'] and data['results']['bindings']:
        image_url = data['results']['bindings'][0].get('image', {}).get('value')
        return image_url
    else:
        return "No image found"

# HTML template
html_template = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Entity Information</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            min-height: 100vh;
            background-color: #f4f4f4;
        }}
        .container {{
            max-width: 800px;
            width: 100%;
            padding: 20px;
            background: white;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 8px;
            text-align: center;
            border: 4px solid black;
            box-sizing: border-box;
        }}
        .title {{
            font-size: 24px;
            font-weight: bold;
            margin-bottom: 20px;
        }}
        .text {{
            font-size: 16px;
            margin: 20px 0;
            max-height: 100px; /* part of the content */
            overflow: hidden;
            transition: max-height 0.3s ease-in-out;
        }}

        .toggle-checkbox {{
            display: none;
        }}

        .toggle-checkbox:checked + .toggle-label + .text {{
            max-height: none; /* Allow the content to expand */
        }}

        .toggle-label {{
            cursor: pointer;
            color: blue;
            font-weight: bold;
            margin-top: 10px;
            display: inline-block;
        }}

        .toggle-checkbox:checked + .toggle-label {{
            color: red;
        }}

        .image {{
            max-width: 100%; /* Ensures the images scale within the container */
            height: auto;
            border-radius: 4px;
        }}

        /* Container for the logo and flag */
        .image-container {{
            display: flex;
            justify-content: center;
            align-items: center;
            gap: 10px; /* Add space between images */
            width: 100%; /* Ensure the container takes up the full width */
            flex-wrap: wrap; /* Allow images to wrap if needed */
        }}

        .image-container img {{
            max-width: 45%; /* Ensure images don’t overflow */
            height: auto;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="image-container">
            <img src="{logo_url}" alt="Entity Logo" class="image">
            <img src="{flag_url}" alt="Country Flag" class="image" style="max-width: 150px; height: 100px; object-fit: contain;">
        </div>

        <h1 class="title">{title}</h1>
        <img src="{car_image}" alt="Car Image" class="image">

        <input type="checkbox" id="toggle" class="toggle-checkbox">
        <label for="toggle" class="toggle-label">Show More</label>

        <div class="text">
            {abstract}
        </div>
    </div>
</body>
</html>
"""

def generate_html(entity_name):
    # Fetch data
    wikidata_uri, abstract = get_wikidata_uri_and_abstract(entity_name.split()[0])
    logo_url = get_logo_from_wikidata(wikidata_uri)
    flag_url = get_flag_from_wikidata(wikidata_uri)
    car_image = get_car_image(entity_name)

    # Fill the template
    filled_html = html_template.format(
        title=entity_name,
        logo_url=logo_url,
        flag_url=flag_url,
        abstract=abstract,
        car_image=car_image
    )

    output_file = f"{entity_name}.html"
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(filled_html)

    print(f"HTML file generated: {os.path.abspath(output_file)}")

# Main execution
if __name__ == "__main__":
    entity_name = input("Enter the entity name: ")

    wikidata_uri, abstract = get_wikidata_uri_and_abstract(entity_name.split()[0])

    if wikidata_uri:
        print("Wikidata URI:", wikidata_uri)
        print("Abstract:", abstract)

        logo_url = get_logo_from_wikidata(wikidata_uri)
        print("Logo URL:", logo_url)

        flag_url = get_flag_from_wikidata(wikidata_uri)
        print("Country Flag URL:", flag_url)

        label = entity_name
        car_image = get_car_image(label)
        print("Car Image URL:", car_image)

        # Generate the HTML
        generate_html(entity_name)
    else:
        print(abstract)


Enter the entity name: Fiat Multipla
Wikidata URI: http://www.wikidata.org/entity/Q27597
Abstract: Fiat Automobiles S.p.A. (UK: /ˈfiːət, -æt/, US: /-ɑːt/, Italian: [ˈfiːat]; originally FIAT, Italian: Fabbrica Italiana Automobili di Torino, lit. 'Italian Automobiles Factory of Turin') is an Italian automobile manufacturer, formerly part of Fiat Chrysler Automobiles, and since 2021 a subsidiary of Stellantis through its Italian division Stellantis Italy. Fiat Automobiles was formed in January 2007 when Fiat S.p.A. reorganized its automobile business, and traces its history back to 1899 when the first Fiat automobile, the Fiat 4 HP, was produced. Fiat Automobiles is the largest automobile manufacturer in Italy. During its more than century-long history, it remained the largest automobile manufacturer in Europe and the third in the world after General Motors and Ford for over 20 years, until the car industry crisis in the late 1980s. In 2013, Fiat S.p.A. was the second largest European aut

# New Section